# 1、热度地图

In [1]:
import jieba
import pymysql
import numpy as np

In [3]:
table_name = 'weibo_table'
create_time_name = 'W_created_at'
host = '10.8.128.205'
port = 23306
user = 'weibo_user'
password = 'ir@DUT#weibo'
db_name = 'weiboDB'
#load dicts
jieba.load_userdict("../data/weibo_data/dicts/SogouLabDic.txt")
jieba.load_userdict("../data/weibo_data/dicts/dict_baidu_utf8.txt")
jieba.load_userdict("../data/weibo_data/dicts/dict_pangu.txt")
jieba.load_userdict("../data/weibo_data/dicts/dict_sougou_utf8.txt")
jieba.load_userdict("../data/weibo_data/dicts/dict_tencent_utf8.txt")
jieba.load_userdict("../data/weibo_data/dicts/my_dict.txt")
stopwords = {}.fromkeys([ line.rstrip() for line in open('../data/weibo_data/dicts/Stopword.txt','r',encoding='utf-8')])

In [4]:
# input：直接从数据库查询 tag =0时输出所有微博计算的热度地图，tag为其他值，则计算tag小时内微博计算的热度地图
# output：输出标准化后的热度地图字典
def get_map_number(tag):
    dict_dir = ''
    jieba.load_userdict(os.path.join(dict_dir,"SogouLabDic.txt"))
    jieba.load_userdict(os.path.join(dict_dir,"dict_baidu_utf8.txt"))
    jieba.load_userdict(os.path.join(dict_dir,"dict_pangu.txt"))
    jieba.load_userdict(os.path.join(dict_dir,"dict_sougou_utf8.txt"))
    jieba.load_userdict(os.path.join(dict_dir,"dict_tencent_utf8.txt"))
    jieba.load_userdict(os.path.join(dict_dir,"my_dict.txt"))
    table_name = 'weibo_table'
    create_time_name = 'W_created_at'
    host = '10.8.128.205'
    port = 23306
    user = 'weibo_user'
    password = 'ir@DUT#weibo'
    db_name = 'weiboDB'
    stopwords = {}.fromkeys([ line.rstrip() for line in open('../data/weibo_data/dicts/Stopword.txt','r',encoding='utf-8')])
    db = pymysql.connect(host = host,port = port,user = user,password = password,db = db_name,charset='utf8',cursorclass = pymysql.cursors.DictCursor)
    cursor = db.cursor()
    if tag != 0:
        # tag小时微博
        sql_1 = "select W_content from " + str(table_name) + " where " + str(create_time_name) + ">=(now() - interval " + str(tag) +" hour)"
    else:
        # 所有微博
        sql_1 = "select W_content from " + str(table_name)
    query = cursor.execute(sql_1)
#     print(query)
    result = []
    result_middle_1 = []
    result_middle = []
    lag = 0
    if query > 0:
        result_middle_1 = cursor.fetchall()
        for i in range(1,len(result_middle_1)):
            result_middle = result_middle_1[i]['W_content']
            seg = jieba.cut(result_middle)
            for i in seg:
                if i not in stopwords:  
                    result.append(i)
    else:
        lag = 1
    db.close()

    # 对筛选时间段内的微博统计地域信息
    if lag != 1: 
        provin = '北京市，天津市，上海市，重庆市，河北省，山西省，辽宁省，吉林省，黑龙江省，江苏省，浙江省，安徽省，福建省，江西省，山东省，河南省，湖北省，湖南省，广东省，海南省，四川省，贵州省，云南省，陕西省，甘肃省，青海省，台湾省，内蒙古自治区，广西壮族自治区，西藏自治区，宁夏回族自治区，新疆维吾尔自治区，香港特别行政区，澳门特别行政区，北京，天津，上海，重庆，河北，山西，辽宁，吉林，黑龙江，江苏，浙江，安徽，福建，江西，山东，河南，湖北，湖南，广东，海南，四川，贵州，云南，陕西，甘肃，青海，台湾，内蒙古，广西，西藏，宁夏，新疆，香港，澳门'
        provin = provin.split('，')
        provin1 = '北京，天津，上海，重庆，河北，山西，辽宁，吉林，黑龙江，江苏，浙江，安徽，福建，江西，山东，河南，湖北，湖南，广东，海南，四川，贵州，云南，陕西，甘肃，青海，台湾，内蒙古，广西，西藏，宁夏，新疆，香港，澳门'
        provin2 = '北京市，天津市，上海市，重庆市，河北省，山西省，辽宁省，吉林省，黑龙江省，江苏省，浙江省，安徽省，福建省，江西省，山东省，河南省，湖北省，湖南省，广东省，海南省，四川省，贵州省，云南省，陕西省，甘肃省，青海省，台湾省，内蒙古自治区，广西壮族自治区，西藏自治区，宁夏回族自治区，新疆维吾尔自治区，香港特别行政区，澳门特别行政区'
        provin1 = provin1.split('，')
        provin2 = provin2.split('，')
        count = {}
        count2 = {}
        # get the weibo numbers of all the cities
        for word in result:
            if word in provin:
                count[word] = count.get(word, 0) + 1
        for i in zip(provin1, provin2):
            count2[i[0]] = count.get(i[0], 0) + count.get(i[1], 0)

        # scale the numbers in 0-100
        values = []
        for i in count2.values():
            values.append(i)
        values = np.array(values)
        try:
            Max = values.max()
            Min = values.min()
            if Max != Min:
                values = ((values - Min) / (Max - Min) * 100).astype(np.int)
            else:
                pass
        except:
            pass
        values = values.tolist()

        # get the final result as [['北京', 46], ['天津', 5],...]
        provin_list = []
        for key, value in zip(provin1, values):
            list1 = [key, value]
            provin_list.append(list1)
        provin_dict = dict()        
        for i in provin_list:        
            provin_dict[i[0]]=i[1]
    else:
        return 'No weibo contains!'
    return provin_dict

In [46]:
result = get_map_number(0)

result = [{'name':str(k),'value':float(result[k])} for k in result]

In [48]:
result

[{'name': '北京', 'value': 70.0},
 {'name': '天津', 'value': 5.0},
 {'name': '上海', 'value': 100.0},
 {'name': '重庆', 'value': 4.0},
 {'name': '河北', 'value': 1.0},
 {'name': '山西', 'value': 2.0},
 {'name': '辽宁', 'value': 4.0},
 {'name': '吉林', 'value': 1.0},
 {'name': '黑龙江', 'value': 1.0},
 {'name': '江苏', 'value': 10.0},
 {'name': '浙江', 'value': 20.0},
 {'name': '安徽', 'value': 2.0},
 {'name': '福建', 'value': 1.0},
 {'name': '江西', 'value': 6.0},
 {'name': '山东', 'value': 9.0},
 {'name': '河南', 'value': 5.0},
 {'name': '湖北', 'value': 0.0},
 {'name': '湖南', 'value': 2.0},
 {'name': '广东', 'value': 4.0},
 {'name': '海南', 'value': 1.0},
 {'name': '四川', 'value': 2.0},
 {'name': '贵州', 'value': 9.0},
 {'name': '云南', 'value': 4.0},
 {'name': '陕西', 'value': 11.0},
 {'name': '甘肃', 'value': 2.0},
 {'name': '青海', 'value': 1.0},
 {'name': '台湾', 'value': 12.0},
 {'name': '内蒙古', 'value': 1.0},
 {'name': '广西', 'value': 0.0},
 {'name': '西藏', 'value': 3.0},
 {'name': '宁夏', 'value': 0.0},
 {'name': '新疆', 'value': 1.0},

# 2、评论热度分析

In [6]:
import requests
import pandas as pd
body = {'wordQuery':'',
        'textQuery':'垃圾分类',
        'volume':1000}
baseUrl = 'http://10.8.128.205:29280/Lawbda/dataWare/1.0.0/weibo/search'
docs = requests.get(baseUrl,params=body)

In [14]:
class esWeiboCommentRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True

    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esWeiboCommentRetrieval, cls).__new__(cls)
        return cls._instance

    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esWeiboCommentRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'weibo-comment-index'

    def do_search(self, wordQuery, textQuery, volume):
        '''
        do_search方法执行具体检索过程
        wordQuery 本应为查询对应微博时所用的检索词 此处暂时不用 目前暂时只检索微博评论数据后续再修改对象为

        volume为每次检索返回的数目
        '''
        queryBody = {
            "query": {
                "match": {
                    "C_content": textQuery
                }
            },
            "from": 0,
            "size": volume,
            "sort": [],
            "aggs": {}
        }
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'C_ID,C_comment_id,C_comment_user_id,C_comment_user_nick_name,C_content,\
        C_weibo_url,C_like_num,C_created_at,C_crawl_time'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs

    def Retrieval(self, wordQuery, textQuery, volume):
        result = self.do_search(wordQuery, textQuery, volume)
        docs = self.format_search(result)
        return docs

In [24]:
# 输入: 评论查询结果docs及参数time_period = ''
# 输出时间段评论数量 period_result  例如  {'2015': 1, '2016': 0, '2017': 21, '2018': 109, '2019': 869}

def get_comment_num(docs,time_period):
    # 按年统计并显示  time_period = 'A'
    # 按季度统计并显示 time_period = 'Q'
    # 按月度统计并显示 time_period = 'M'
    # 按周统计并显示 time_period = 'W'
    # 按天统计并显示 time_period = 'D'
    # 按小时统计并显示 time_period = '12h'
    # 按分钟统计并显示 time_period = '60min'
    data=docs
    time_list =[i['C_created_at'].replace('T',' ') for i in docs]
    time_result={'date':time_list,'num':1}
    data=pd.DataFrame(time_result)
    data['date'] = pd.to_datetime(data['date'])
    df = data.set_index('date')
    result = df.resample(time_period).sum().to_period(time_period)
    num_sum = [int(np.squeeze(i)) for i in result.values]
    date_split = [str(i) for i in result.index]
    period_result = dict(zip(date_split, num_sum))
    return period_result

In [21]:
esObj = esWeiboCommentRetrieval(host='10.8.128.205',port=49200)

docs = esObj.Retrieval(wordQuery='',textQuery='垃圾分类',volume=100)

result = get_comment_num(docs, time_period = '2m')

In [28]:
result

{'2017-12': 3,
 '2018-02': 0,
 '2018-04': 4,
 '2018-06': 2,
 '2018-08': 5,
 '2018-10': 0,
 '2018-12': 6,
 '2019-02': 1,
 '2019-04': 0,
 '2019-06': 1,
 '2019-08': 78}

In [29]:
result = [{'name':str(k),'value':float(result[k])} for k in result]

# 3、返回查询词对应微博

In [9]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch

In [10]:
class esWeiboTweetRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esWeiboTweetRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esWeiboTweetRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'weibo-tweet-index'

    def do_search(self, W_search_query, W_content, volume):
        '''
        do_search方法执行具体检索过程
        wordQuery 本应为查询对应微博时所用的检索词 此处暂时不用 目前暂时只检索微博评论数据后续再修改对象为
        
        volume为每次检索返回的数目
        '''
        queryBody = {
        "query": {
        "bool": {
        "must": [ ],
        "must_not": [ ],
        "should": [
        {
        "query_string": {
        "default_field": "W_content",
        "query": W_content
        }
        }
        ,
        {
        "query_string": {
        "default_field": "W_search_query",
        "query": W_search_query
        }
        }
        ]
        }
        },
        "from": 0,
        "size": volume,
        "sort": [ ],
        "aggs": { }
        }
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'W_ID,W_weibo_id,W_search_query,W_nick_name,W_weibo_url,W_created_at,\
        W_like_num,W_repost_num,W_comment_num,W_content,W_user_id,W_image_url,W_video_url,W_tool,W_location,\
        W_location_map_info,W_origin_weibo_url,W_origin_weibo_content,W_crawl_time'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = list(dict_filter_by_keys(doc) for doc in docs)
        return docs
    def Retrieval(self, W_search_query, W_content, volume):
        result = self.do_search(W_search_query, W_content, volume)
        docs = self.format_search(result)
        return docs

In [11]:
eee = esWeiboTweetRetrieval(host='10.8.128.205',port=49200,)

result= eee.Retrieval(W_search_query='#垃圾分类#', W_content='北京',volume=1000)

In [13]:
# input 微博检索结果
def get_query_weibo(result):
    return result

In [ ]:
get_query_weibo(result)

# 4、情感分析

In [27]:
import pickle
from jieba import analyse
import pickle
import os
import jieba
from jieba import analyse
from snownlp import SnowNLP
from snownlp.sentiment import Sentiment
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt

In [31]:
class esWeiboTweetRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esWeiboTweetRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esWeiboTweetRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'weibo-tweet-index'

    def do_search(self, W_search_query, W_content, volume):
        '''
        do_search方法执行具体检索过程
        wordQuery 本应为查询对应微博时所用的检索词 此处暂时不用 目前暂时只检索微博评论数据后续再修改对象为
        
        volume为每次检索返回的数目
        '''
        queryBody = {
        "query": {
        "bool": {
        "must": [ ],
        "must_not": [ ],
        "should": [
        {
        "query_string": {
        "default_field": "W_content",
        "query": W_content
        }
        }
        ,
        {
        "query_string": {
        "default_field": "W_search_query",
        "query": W_search_query
        }
        }
        ]
        }
        },
        "from": 0,
        "size": volume,
        "sort": [ ],
        "aggs": { }
        }
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'W_ID,W_weibo_id,W_search_query,W_nick_name,W_weibo_url,W_created_at,\
        W_like_num,W_repost_num,W_comment_num,W_content,W_user_id,W_image_url,W_video_url,W_tool,W_location,\
        W_location_map_info,W_origin_weibo_url,W_origin_weibo_content,W_crawl_time'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = list(dict_filter_by_keys(doc) for doc in docs)
        return docs
    def Retrieval(self, W_search_query, W_content, volume):
        result = self.do_search(W_search_query, W_content, volume)
        docs = self.format_search(result)
        return docs

In [38]:
# input： 微博检索结果
# output： {
#         dict_senti          微博情感得分
#         pos_content[0:5]    正向微博前5条
#         neg_content[0:5]    负向微博前5条
# }

def get_weibo_senti(result):
    happy = ["PA","PE"]#乐
    like = ["PD","PH","PG","PB","PK"]#好
    anger = ["NA"]#怒
    sad = ["NB","NJ","NH","PF"]#哀
    afraid = ["NI","NC","NG"]#惧
    boring = ["NE","ND","NN","NK","NL"]#恶
    suprise = ["PC"]#惊
    emotion = pd.read_excel('../data/weibo_data/senti onthology.xlsx', header=0, encoding='utf8')
    words = emotion["词语"].tolist()
    emo = emotion["情感分类"].tolist()
    core = emotion["强度"].tolist()
    pos_neg = emotion["极性"].tolist()
    res = [i['W_content'] for i in result]

    num_happy=0
    num_like =0
    num_anger=0
    num_sad = 0
    num_afraid = 0
    num_boring = 0
    num_suprise = 0
    num_pos=0
    num_neg=0
    happy_content = []
    like_content = []
    anger_content = []
    sad_content = []
    afraid_content = []
    boring_content = []
    suprise_content = []
    pos_content = []
    neg_content = []
    if len(res)>500:
        N=500
    else:
        N = len(res)
    for i in res[:N]: 
        sore_happy=0
        sore_like =0
        sore_anger=0
        sore_sad = 0
        sore_afraid = 0
        sore_boring = 0
        sore_suprise = 0
        sore_pos=0
        sore_neg=0
        if len(i)>4:
            for word in jieba.lcut(i):
                if word in emotion["词语"].tolist():
                    aa = emotion.loc[emotion['词语']==word]
                    if aa["情感分类"].all() in happy:                  
                         sore_happy= aa["强度"].tolist()[0]+sore_happy
                    elif aa["情感分类"].all() in like:
                        sore_like = aa["强度"].tolist()[0]+sore_like
                    elif aa["情感分类"].all() in anger:
                        sore_anger = aa["强度"].tolist()[0]+sore_anger
                    elif aa["情感分类"].all() in sad:
                        sore_sad = aa["强度"].tolist()[0]+sore_sad
                    elif aa["情感分类"].all() in afraid:
                        sore_afraid = aa["强度"].tolist()[0]+sore_afraid
                    elif aa["情感分类"].all() in boring:
                        sore_boring = aa["强度"].tolist()[0]+sore_boring
                    elif aa["情感分类"].all() in suprise:
                        sore_suprise = aa["强度"].tolist()[0]+sore_suprise
                    if aa["极性"].tolist()[0] == 1:
                        sore_pos = aa["强度"].tolist()[0]+sore_pos
                    elif aa["极性"].tolist()[0] == 2:
                        sore_neg = aa["强度"].tolist()[0]+sore_neg
            emo=[]
            emo.append(sore_happy)
            emo.append(sore_like)
            emo.append(sore_anger)
            emo.append(sore_sad)
            emo.append(sore_afraid)
            emo.append(sore_boring)
            emo.append(sore_suprise)
            senti = []
            senti.append(sore_pos)
            senti.append(sore_neg)
            if max(emo) == sore_happy:
                happy_content.append(i)
                num_happy = num_happy+1
            elif max(emo) == sore_like:
                like_content.append(i)
                num_like = num_like+1
            elif max(emo) == sore_anger:
                anger_content.append(i)
                num_anger = num_anger+1
            elif max(emo) == sore_sad:
                sad_content.append(i)
                num_sad = num_sad+1
            elif max(emo) == sore_afraid:
                afraid_content.append(i)
                num_afraid = num_afraid+1
            elif max(emo) == sore_boring:
                boring_content.append(i)
                num_boring = num_boring+1
            elif max(emo) == sore_suprise:
                suprise_content.append(i)
                num_suprise = num_suprise+1
            if max(senti) == sore_pos:
                pos_content.append(i)
                num_pos = num_pos+1
            else:
                neg_content.append(i)
                num_neg = num_neg+1
            sum1 = num_happy+num_like+num_anger+num_sad+num_afraid+num_boring+num_suprise
            dict_emo = {"happy":round(num_happy/sum1,3),"like":round(num_like/sum1,3),"anger":round(num_anger/sum1,3),
                        "sad":round(num_sad/sum1,3),"afraid":round(num_afraid/sum1,3),"boring":round(num_boring/sum1,3),
                        "suprise":round(num_suprise/sum1,3)}
            dict_senti = {"positive":round(num_pos/(num_pos+num_neg),3),"negtive":round(num_neg/(num_pos+num_neg),3)}
            content_emo = {"happy":happy_content,"like":like_content,"anger":anger_content,
                        "sad":sad_content,"afraid":afraid_content,"boring":boring_content,
                        "suprise":suprise_content}
            content_senti = {"positive":pos_content,"negtive":neg_content}
    return dict_senti, pos_content[0:5], neg_content[0:5]

In [32]:
esObj = esWeiboTweetRetrieval(host='10.8.128.205',port=49200)

docs = esObj.Retrieval(W_search_query='垃圾分类',W_content='垃圾分类',volume=100)

dict_senti, pos_content, neg_content = get_weibo_senti(docs)

result = {'dict_senti':dict_senti,
         'pos_content':pos_content,
         'neg_content':neg_content}

In [49]:
dict_senti

{'negtive': 0.11, 'positive': 0.89}

In [50]:
pos_content

['#垃圾分类# 北京市法制宣传教育领导小组办公室、北京市司法局：“不分类 环境恶化、资源浪费”，“厨余垃圾→堆肥”。很想问问：现在北京厨余垃圾多少堆肥了？效率、效果、去向如何？能不能将混合垃圾和较纯厨余分开堆肥，对比一下？@北京市市政市容委@卫潘明@张红樱@立雯nu@垃圾战斗机FON@冯永锋',
 '#北京将立法约束垃圾分类#北京将立法明确垃圾分类个人责任，混合投放垃圾将被罚！#垃圾分类# 王希希ss的微博视频',
 '【北京拟对垃圾不分类的个人罚款，视频科普#干湿垃圾怎么区分# [思考]】日前，北京市城市管理委主任孙新军在做客节目时透露，《北京市生活垃圾管理条例》修订工作已列入2018-2020年的立法规划，新修订的条例拟对个人明确垃圾分类责任，混合投放垃圾最高要处以200元罚款，北京也不会低于这个数。对于北京针对垃圾分类的骤然提速，很多网友非常懵：各类垃圾到底怎么分？#垃圾分类# 视频科普：http://t.cn/AipS8vSa',
 '#你是个什么垃圾北京版# 一个#垃圾分类# 连续上热门得有一周了吧，还以为上海人民先替全国人民受难会坚持一年半载，没想到全国推行这么快，北京也要实施了[允悲] 关键是北京、上海标准还不一样，像我们这种一年有四个月在上海出差的人，要背两套规定啊[允悲][允悲][允悲][允悲]  #北京将推动垃圾分类立法#',
 '#垃圾分类# 上海北京步入垃圾分类强制时代，来自河北的你做好准备了吗？ [组图共2张]']

In [51]:
neg_content

['【[话筒]提前转存！北京将推动垃圾分类立法 分类标准与上海不完全一致】#垃圾分类# 日前，记者从北京市城管委了解到，北京将推动垃圾分类立法，罚款不低于上海。但北京的垃圾分类标准与上海并不完全一致，分为厨余垃圾、有害垃圾、其它垃圾及可回收物。具体怎么分？戳视频↓  南昌高新区发布的微博视频',
 '来自网络评论：《北京青年》就是一个北京老年用一群北京中年来演绎空想的北京青年的故事。妙哉！',
 '北京政府真是无耻到了极点，估计过几年来北京就得签证了！！！！！',
 '#垃圾分类 卡路里#闻圾起舞寓教于乐[给力] 北京人民也在瑟瑟发抖[允悲][允悲][允悲]谈小培在旅途的秒拍视频',
 'http://t.cn/Ai0SCtdm 2017年北京生活垃圾清运量为924.77万吨，如果用载重5吨的卡车来运送，两辆一排，可以从云南西双版纳排到哈尔滨。目前我国城市垃圾处理最主要的方式是填埋与焚烧，但都存在一定的弊端，假设北京的生活#垃圾分类# 妥善，每年因垃圾致癌的人数将从241人降低至182人。@第一财经日报 [组图共5张]']